In [1]:
import sys
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import data
import preprocess_utils.session2vec as sess2vec
import preprocess_utils.sessions_to_predict as sess2predict
from preprocess_utils.last_clickout_indices import find as find_last_clickout_indices
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None

Using TensorFlow backend.


In [3]:
mode = 'full'
cluster = 'cluster_recurrent'

In [4]:
train_df = data.train_df('full', 'cluster_recurrent')

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
test_df = data.test_df('full', 'cluster_recurrent')

## Feature

In [6]:
train_df.drop(['city','device','frequence'], axis=1, inplace=True)
test_df.drop(['city','device','frequence'], axis=1, inplace=True)

In [9]:
df = pd.concat([train_df, test_df])

In [15]:
df.head(8)

,user_id,session_id,timestamp,step,action_type,reference,platform,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,NaN,NaN,NaN
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,NaN,NaN,NaN
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,NaN,NaN,NaN
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,NaN,NaN,NaN
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,NaN,NaN,NaN
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...


In [10]:
# find the last clickout rows
idxs = find_last_clickout_indices(df)

In [29]:
res_df = df[['reference','impressions']].loc[idxs]
res_df.shape

(1102508, 2)

In [30]:
res_df[res_df.reference.isnull()]

,reference,impressions
19023774,NaN,923407|1729121|1050792|97171|353141|106315|218...
16963755,NaN,2213014|3184842|10213134|4504242|4486372|38120...
16963756,NaN,3812004|3505150|3202894|2292254|1984229|222789...
18930874,NaN,22721|3067559|16121|22727|22854|22819|22764|14...
16010186,NaN,507861|2176280|1669587|502066|1352530|4342348|...
19519827,NaN,157710|1287958|160577|1618677|7231396|319866|4...
16271468,NaN,102540|33191|2285010|2547840|20144|2808050|448...
17001782,NaN,2184212|2756867|2909994|4063406|3874514|950324...
19417495,NaN,5416250|7795438|3926654|8989706|6797450|722521...
17921071,NaN,32256|32230|5411980|32246|32233|32235|5592656|...


In [31]:
res_df = res_df.dropna(subset=['reference']).astype({'reference':'int'})
res_df.shape

(848935, 2)

In [32]:
# create impressions list
res_df['impressions_list'] = res_df['impressions'].str.split('|').apply(lambda x: list(map(int,x)))
res_df.drop('impressions', axis=1, inplace=True)

In [35]:
label_series = np.zeros(res_df.shape[0], dtype='int8')
k = 0
for row in tqdm(zip(res_df['reference'], res_df['impressions_list'])):
    ref = row[0]
    impress = row[1]
    if ref in impress:
        label_series[k] = impress.index(ref)
    k += 1

In [36]:
res_df['label'] = label_series
res_df

,reference,impressions_list,label
8321598,950829,"[104716, 1240220, 106938, 9445554, 104879, 950...",5
12335024,2195060,"[2195060, 3053334, 4073410, 1333107, 7001704, ...",0
27778,110985,"[2632452, 5747484, 110985, 84220, 3752262, 851...",2
7333077,1439375,"[1439375, 120148, 115949, 741941, 130127, 4036...",0
10577356,6776722,"[6776722, 9772048, 9017330, 3463608, 8147814, ...",0
464545,2762974,[2762974],0
13208141,452021,"[2786838, 5757530, 477876, 403596, 4247892, 67...",8
13933216,1949601,"[9857656, 1352530, 3810222, 6753996, 1949601, ...",4
11769883,3480564,"[3480564, 1306892, 1796781, 781196, 4022130, 1...",0
9588794,5665184,"[1713885, 4122822, 5194300, 405701, 1775983, 4...",18


In [8]:
add_reference_labels(train_df.loc[idxs], mode='full')

session_label feature read


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,ref_0,ref_1,ref_2,ref_3,ref_4,ref_5,ref_6,ref_7,ref_8,ref_9,ref_10,ref_11,ref_12,ref_13,ref_14,ref_15,ref_16,ref_17,ref_18,ref_19,ref_20,ref_21,ref_22,ref_23,ref_24
8321598,0001VQMGUI65,466687f1875ee,1541076401,17,clickout item,950829,BR,"Brasília, Brazil",desktop,Sort by Price,104716|1240220|106938|9445554|104879|950829|95...,47|47|47|47|48|49|50|49|49|50|50|51|51|51|51|5...,1.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12335024,0003QTCX5MJX,6e81f1bdeb75d,1541441472,4,clickout item,2195060,TR,"Istanbul, Turkey",mobile,Sort by Price,2195060|3053334|4073410|1333107|7001704|216871...,10|18|18|19|20|20|20|21|22|22|22|22|23|23|23|2...,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27778,0004IOZI7CKF,0146f7cb014ba,1541266868,5,clickout item,110985,DE,"Valencia, Spain",tablet,NaN,2632452|5747484|110985|84220|3752262|85103|852...,75|62|113|91|106|115|75|68|81|86|36|66|86|159|...,1.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7333077,0004WCFRV3FB,b47e1ccca2b97,1541421814,2,clickout item,1439375,DE,"Riezlern, Austria",mobile,NaN,1439375|120148|115949|741941|130127|4036512|18...,209|178|279|106|203|194|90|173|102|154|84|733|...,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10577356,0006W0R5A5V8,85501a9f27263,1541385464,1,clickout item,6776722,CO,"Tocaima, Colombia",desktop,NaN,6776722|9772048|9017330|3463608|8147814|951364...,53|37|33|138|41|37|45|27|62|29|63|33|43|21,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
464545,0008B0X0HC39,1b703eda68d85,1541445203,1,clickout item,2762974,US,"Coron, Philippines",desktop,NaN,2762974,494,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13208141,0008BO33KUQ0,d4c5cc7f469d7,1541298200,12,clickout item,452021,BR,"Aquiraz, Brazil",mobile,NaN,2786838|5757530|477876|403596|4247892|6797610|...,175|167|149|88|89|70|226|76|61|107|106|88|83|9...,1.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13933216,0008BO33KUQ0,eaae649fbd8c9,1541337472,4,clickout item,1949601,BR,"Gramado, Brazil",mobile,NaN,9857656|1352530|3810222|6753996|1949601|313320...,178|298|138|76|80|145|79|91|149|192|131|377|18...,1.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11769883,0008W02TLRNR,c39aa6a976e9b,1541305941,1,clickout item,3480564,US,"Cúcuta, Colombia",mobile,NaN,3480564|1306892|1796781|781196|4022130|1626615...,44|39|46|29|28|38|49|33|14|23|19|38|8|16|34|29...,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9588794,000EKQEFMW6S,115f3370c7c43,1541277856,2,clickout item,5665184,MX,"San Miguel de Allende, Mexico",desktop,NaN,1713885|4122822|5194300|405701|1775983|42716|6...,569|166|132|88|76|257|260|70|127|119|142|53|33...,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [25]:
train_df.loc[idxs].merge(feature.drop(['user_id','session_id'],axis=1), how='left', left_index=True, right_index=True)

,user_id,session_id,timestamp,step,action_type,reference,platform,current_filters,impressions,prices,label
8321598,0001VQMGUI65,466687f1875ee,1541076401,17,clickout item,950829,BR,Sort by Price,104716|1240220|106938|9445554|104879|950829|95...,47|47|47|47|48|49|50|49|49|50|50|51|51|51|51|5...,5
12335024,0003QTCX5MJX,6e81f1bdeb75d,1541441472,4,clickout item,2195060,TR,Sort by Price,2195060|3053334|4073410|1333107|7001704|216871...,10|18|18|19|20|20|20|21|22|22|22|22|23|23|23|2...,0
27778,0004IOZI7CKF,0146f7cb014ba,1541266868,5,clickout item,110985,DE,NaN,2632452|5747484|110985|84220|3752262|85103|852...,75|62|113|91|106|115|75|68|81|86|36|66|86|159|...,2
7333077,0004WCFRV3FB,b47e1ccca2b97,1541421814,2,clickout item,1439375,DE,NaN,1439375|120148|115949|741941|130127|4036512|18...,209|178|279|106|203|194|90|173|102|154|84|733|...,0
10577356,0006W0R5A5V8,85501a9f27263,1541385464,1,clickout item,6776722,CO,NaN,6776722|9772048|9017330|3463608|8147814|951364...,53|37|33|138|41|37|45|27|62|29|63|33|43|21,0
464545,0008B0X0HC39,1b703eda68d85,1541445203,1,clickout item,2762974,US,NaN,2762974,494,0
13208141,0008BO33KUQ0,d4c5cc7f469d7,1541298200,12,clickout item,452021,BR,NaN,2786838|5757530|477876|403596|4247892|6797610|...,175|167|149|88|89|70|226|76|61|107|106|88|83|9...,8
13933216,0008BO33KUQ0,eaae649fbd8c9,1541337472,4,clickout item,1949601,BR,NaN,9857656|1352530|3810222|6753996|1949601|313320...,178|298|138|76|80|145|79|91|149|192|131|377|18...,4
11769883,0008W02TLRNR,c39aa6a976e9b,1541305941,1,clickout item,3480564,US,NaN,3480564|1306892|1796781|781196|4022130|1626615...,44|39|46|29|28|38|49|33|14|23|19|38|8|16|34|29...,0
9588794,000EKQEFMW6S,115f3370c7c43,1541277856,2,clickout item,5665184,MX,NaN,1713885|4122822|5194300|405701|1775983|42716|6...,569|166|132|88|76|257|260|70|127|119|142|53|33...,18


In [26]:
joined = _

In [29]:
joined.astype({'label':'int'})

,user_id,session_id,timestamp,step,action_type,reference,platform,current_filters,impressions,prices,label
8321598,0001VQMGUI65,466687f1875ee,1541076401,17,clickout item,950829,BR,Sort by Price,104716|1240220|106938|9445554|104879|950829|95...,47|47|47|47|48|49|50|49|49|50|50|51|51|51|51|5...,5
12335024,0003QTCX5MJX,6e81f1bdeb75d,1541441472,4,clickout item,2195060,TR,Sort by Price,2195060|3053334|4073410|1333107|7001704|216871...,10|18|18|19|20|20|20|21|22|22|22|22|23|23|23|2...,0
27778,0004IOZI7CKF,0146f7cb014ba,1541266868,5,clickout item,110985,DE,NaN,2632452|5747484|110985|84220|3752262|85103|852...,75|62|113|91|106|115|75|68|81|86|36|66|86|159|...,2
7333077,0004WCFRV3FB,b47e1ccca2b97,1541421814,2,clickout item,1439375,DE,NaN,1439375|120148|115949|741941|130127|4036512|18...,209|178|279|106|203|194|90|173|102|154|84|733|...,0
10577356,0006W0R5A5V8,85501a9f27263,1541385464,1,clickout item,6776722,CO,NaN,6776722|9772048|9017330|3463608|8147814|951364...,53|37|33|138|41|37|45|27|62|29|63|33|43|21,0
464545,0008B0X0HC39,1b703eda68d85,1541445203,1,clickout item,2762974,US,NaN,2762974,494,0
13208141,0008BO33KUQ0,d4c5cc7f469d7,1541298200,12,clickout item,452021,BR,NaN,2786838|5757530|477876|403596|4247892|6797610|...,175|167|149|88|89|70|226|76|61|107|106|88|83|9...,8
13933216,0008BO33KUQ0,eaae649fbd8c9,1541337472,4,clickout item,1949601,BR,NaN,9857656|1352530|3810222|6753996|1949601|313320...,178|298|138|76|80|145|79|91|149|192|131|377|18...,4
11769883,0008W02TLRNR,c39aa6a976e9b,1541305941,1,clickout item,3480564,US,NaN,3480564|1306892|1796781|781196|4022130|1626615...,44|39|46|29|28|38|49|33|14|23|19|38|8|16|34|29...,0
9588794,000EKQEFMW6S,115f3370c7c43,1541277856,2,clickout item,5665184,MX,NaN,1713885|4122822|5194300|405701|1775983|42716|6...,569|166|132|88|76|257|260|70|127|119|142|53|33...,18


## Feature testing

In [18]:
def session_label(df, mode):
    tqdm.pandas()

    if mode != 'full':
        df = pd.concat([data.train_df(self.mode), data.test_df(self.mode)])
    else:
        df = data.train_df(mode)

    # find the last clickout rows
    idxs = find_last_clickout_indices(df)

    res_df = df[['user_id','session_id','reference','impressions']].loc[idxs]
    res_df['impression_list'] = 

    def add_label(row):
        ref = row.reference
        #if pd.isnull(ref):
        #    return np.NaN
        impress = list(map(int, row['impressions'].split('|')))

        if ref in impress:
            return impress.index(ref)
        else:
            return 0

    res_df = res_df.astype({'reference':'int'})
    res_df['label'] = res_df.progress_apply(add_label, axis=1)
    res_df = res_df.dropna().astype({'label':'int'})

    return res_df.drop(['reference','impressions'], axis=1)

In [19]:
feature = session_label(train_df, mode='full')

Loading full train_df, it will take a while..


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user_id,session_id,label
8321598,0001VQMGUI65,466687f1875ee,5
12335024,0003QTCX5MJX,6e81f1bdeb75d,0
27778,0004IOZI7CKF,0146f7cb014ba,2
7333077,0004WCFRV3FB,b47e1ccca2b97,0
10577356,0006W0R5A5V8,85501a9f27263,0
464545,0008B0X0HC39,1b703eda68d85,0
13208141,0008BO33KUQ0,d4c5cc7f469d7,8
13933216,0008BO33KUQ0,eaae649fbd8c9,4
11769883,0008W02TLRNR,c39aa6a976e9b,0
9588794,000EKQEFMW6S,115f3370c7c43,18


In [21]:
feature_indices = set(feature.index.values)

In [23]:
feature_indices.difference(set(idxs))

set()

In [5]:
len(idxs)

826842

In [10]:
from extract_features.rnn.session_label import SessionLabel

In [11]:
feat = SessionLabel('full').read_feature()

session_label feature read


In [13]:
test2_df, sessions_not_to_predict = sess2predict.find(test_df)

KeyboardInterrupt: 

In [13]:
last_click_train2 = find_last_clickout_indices(sessions_not_to_predict)

In [14]:
len(last_click_train2)

22106

In [15]:
len(set(last_click_train2).intersection(set(feat.index)))

22106

## Dataset creation

In [6]:
test_df = data.test_df(mode, cluster)

print('Moving sessions not to be predicted from test to train...')
test_df, sessions_not_to_predict = sess2predict.find(test_df)
test_df = test_df.sort_values(['user_id','session_id','timestamp','step'])

#if not only_test:
## ======== TRAIN ======== ##
train_df = data.train_df(mode, cluster)
train_df = train_df.append(sessions_not_to_predict).sort_values(['user_id','session_id','timestamp','step'])

Moving sessions not to be predicted from test to train...


In [7]:
df = sess2vec.pad_sessions(train_df, max_session_length=3)

3990846it [06:54, 9633.59it/s] 


In [8]:
# create Y
Y_COLUMNS = ['user_id','session_id','timestamp','step']
df_Y = df[Y_COLUMNS]

# take only the target rows from y
df_Y = df_Y.iloc[np.arange(-1,len(df_Y),3)[1:]]

# add the reference classes
print('Adding references classes...')
df_Y = sess2vec.add_reference_labels(df_Y, mode=mode)

Adding references classes...
session_label feature read


In [12]:
idx_ok = set(df_Y.index).intersection(set(feat.index))
len(idx_ok)

848947

In [13]:
len(df_Y.index)

848947

In [14]:
set(df_Y.index).difference(set(idx_ok))

set()

In [21]:
train_df.loc[[501301]]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
501301,PDU6XRFDCS11,7e5ec0a512233,1541115164,2,clickout item,43174,MX,"Acapulco, Mexico",mobile,NaN,740596|43174|42334|34343|34350|42446|34337|948...,47|60|66|63|3369|39|72|70|23|35|11|14|54|52|28...,1.0


In [23]:
train_df[(train_df.user_id == 'PDU6XRFDCS11') & (train_df.session_id == '7e5ec0a512233')]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
501300,PDU6XRFDCS11,7e5ec0a512233,1541115043,1,clickout item,740596,MX,"Acapulco, Mexico",mobile,NaN,740596|43174|42334|34343|34350|42446|34337|948...,47|60|66|63|3369|39|72|70|23|35|11|14|54|52|28...,1.0
501301,PDU6XRFDCS11,7e5ec0a512233,1541115164,2,clickout item,43174,MX,"Acapulco, Mexico",mobile,NaN,740596|43174|42334|34343|34350|42446|34337|948...,47|60|66|63|3369|39|72|70|23|35|11|14|54|52|28...,1.0


In [24]:
test_df[(test_df.user_id == 'PDU6XRFDCS11') & (test_df.session_id == '7e5ec0a512233')]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
18927010,PDU6XRFDCS11,7e5ec0a512233,1541562659,1,search for poi,Costera Miguel Alemán,MX,"Acapulco, Mexico",mobile,Beach|Deals + Beach (MX)|Direct beach access,NaN,NaN,1.0
18927011,PDU6XRFDCS11,7e5ec0a512233,1541562678,2,interaction item info,43174,MX,"Acapulco, Mexico",mobile,NaN,NaN,NaN,1.0
18927012,PDU6XRFDCS11,7e5ec0a512233,1541562703,3,interaction item deals,43174,MX,"Acapulco, Mexico",mobile,NaN,NaN,NaN,4.0
18927016,PDU6XRFDCS11,7e5ec0a512233,1541562725,7,clickout item,NaN,MX,"Acapulco, Mexico",mobile,Beach|Deals + Beach (MX)|Direct beach access,43174|34338|34337|100302|34346|1538159|995511|...,61|116|73|122|60|56|33|56|40|48|16|32|38|39|67...,1.0


In [37]:
df = sess2vec.pad_sessions(train_df.head(1240), max_session_length=3)

778it [00:00, 7775.97it/s]

In [40]:
df['frequence'].sum()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
-1,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0
-1,0,0,0,0,0,0,0,0,0,0,0,0,0


## Old feature

In [25]:
def extract_feature(self):
    tqdm.pandas()
    """
    Train and test cannot be concatenated because there are some sessions that
    are splitted and they have a first part in train and the last in the test.
    In those cases, the label will be only one (since they will be treated as
    one session by 'find_last_clickout_indices' function) but they must have
    2 different labels (1 for the train half, 1 for the test half)
    """
    df = pd.concat([data.train_df(self.mode), data.test_df(self.mode)])

    # find the last clickout rows
    idxs = find_last_clickout_indices(df)

    res_df = df[['user_id','session_id','reference','impressions']].loc[idxs]
    # remove the test sessions with reference NaN
    res_df = res_df.dropna(subset=['reference']).astype({'reference':'int'})
    # create impressions list
    res_df['impressions_list'] = res_df['impressions'].str.split('|').apply(lambda x: list(map(int,x)))
    res_df.drop('impressions', axis=1, inplace=True)

    label_series = np.zeros(res_df.shape[0], dtype='int8')
    # iterate over the rows
    k = 0
    for row in tqdm(zip(res_df['reference'], res_df['impressions_list'])):
        ref = row[0]
        impress = row[1]
        if ref in impress:
            label_series[k] = impress.index(ref)
        k += 1
    # add the new column
    res_df['label'] = label_series

    return res_df.drop(['reference','impressions_list'], axis=1)